<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install firebase



In [71]:
from ipywidgets.widgets.widget_button import ButtonStyle
#@title
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from ipywidgets import VBox, HBox, HTML, Layout, Box, Label
import matplotlib.pyplot as plt
import math
######## GLOBAL VARIABLES
questionsFromDB = []
btn_go_back = NULL

# Points
points = 0
quantity_of_questions=0
quantity_of_correct_questions=0
list_of_stats = []  # will be filled with number of attempts for EACH question

currentSelectedAns = ""
finalSelectedBtn = widgets.Button(description="") # null object
question = ""
randomQuestionAndAnswer = NULL
answers = []
buttonsLst = []
questionsHistory = set() # saving the 'old' questions for non repeating. Can also be done by removing those question from all DB list (but i think it's not preferred)
FIREBASE_PATH = 'https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/'
style = {'description_width': '150px'} 

# Admin Form
cb_correctAnswerId = NULL
tb_question= NULL
tb_answer1 = NULL
tb_answer2 = NULL
tb_answer3 = NULL
tb_answer4 = NULL
offset = 0

########## Functions
def get_all_database():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  res = FBconn.get('/Questions/',None)

  if res:
    for key in res:
      if key is not None: 
        questionsFromDB.append(key)

# Define a function to save the selected answer

def get_question_and_answers_dictionary():
        global randomQuestionAndAnswer, answers, question, radioButtonLst

        randomQuestionAndAnswer = NULL # we want to generate new item...
        if (questionsFromDB):
          while randomQuestionAndAnswer == NULL:
            randomQuestionAndAnswer = random.choice(questionsFromDB)
            if str(randomQuestionAndAnswer['question']) in questionsHistory:
              randomQuestionAndAnswer = NULL
        else:
          return

        if randomQuestionAndAnswer != NULL:            
          question = str(randomQuestionAndAnswer['question'])
          questionsHistory.add(question)
          answers = [str(ans) for ans in randomQuestionAndAnswer['answers']]
          answers = answers[1:]

def show_statistics():
  global list_of_stats
  # Create a figure and axis object
  fig, ax = plt.subplots()
  print(list_of_stats)
  # Plot the data as a line plot
  ax.plot(list_of_stats)

  # Set the x-axis and y-axis ticks to round numbers
  ax.set_yticks(range(1, 5))
  ax.set_xticks(range(1, len(list_of_stats)+1))

  # Set the title and labels for the plot
  ax.set_title("Your game statistics")
  ax.set_ylabel("Number of attempts")
  ax.set_xlabel("Question #")

  # Create a widget to display the plot
  output_widget = widgets.Output()

  # Display the widget
  display(output_widget)

  # Render the plot in the widget
  with output_widget:
      plt.show()
  

def game_over():
  """
  End game - show the score and statistics
  """
  output.clear()
  total_score_to_earn = 3*len(questionsFromDB)
  display(IPython.display.HTML(f"<h2>Game over!</h2>"))
  display(IPython.display.HTML(f"<h4>Your score</h4><br/><strong>Points:</strong> {points}"+
  f"<strong> Correct answers:</strong> {quantity_of_correct_questions}"
  f"<br/><strong>You could earn up to</strong> {total_score_to_earn} points."
  ))
  good_words = ""
  if total_score_to_earn < 2:
    good_words = "You are AWESOME!"
  elif total_score_to_earn - points < 15:
    good_words = "You are great! Try again to beat yourself "
  else:
    good_words = "You are good, but need more practice. try out our courses and play again"
  display(IPython.display.HTML(f"<h4>{good_words}</h4>"))
  show_statistics()

# on_click of answer button
def save_answer(b):
  global currentSelectedAns, finalSelectedBtn
  currentSelectedAns = str(b.description)  # save the answer to compare
  finalSelectedBtn = b  # save the button object to mark it later
  on_answer_button_clicked(b)
  update_total_questions_quantity()
  if 3 == len(questionsHistory): # LIDOR - CHANGE this to len(questionsFromDB) instead of 3
      game_over()
      return

def create_and_show_elements():
  global randomQuestionAndAnswer, answers, question, radioButtonLst
  display(IPython.display.HTML(f"<h2>The question is: {question}</h2>"))
  # display(IPython.display.Markdown(f"**Points:** {points}"))
  # correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
  # total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
  # points_label = IPython.display.Markdown(f"**Points:** {points}")
  points_label = HTML(f"<strong>Points:</strong> {points}")
  correct_label = HTML(f"<strong>Correct Questions:</strong> {quantity_of_correct_questions}")
  total_label = HTML(f"<strong>Out Of:</strong> {quantity_of_questions}")

  display(HBox([points_label, correct_label, total_label]))

  buttonsLst = [widgets.Button(description = answer, layout = widgets.Layout( width='auto', height='auto'),
                    style={'white_space': 'pre-wrap'}
                  ) for answer in answers]

  for btn in buttonsLst:
    btn.on_click(save_answer)
    btn.style.button_color = 'orange'

  left_box = VBox([buttonsLst[0], buttonsLst[1]])
  right_box = VBox([buttonsLst[2], buttonsLst[3]])
  display(HBox([left_box, right_box]))


# refresh view -> get new question
def generate_new_question():
  global show_go_back
  output.clear()
  get_question_and_answers_dictionary()
  create_and_show_elements()
  btn_go_back.description = "Exit game"
  show_go_back(create_menu_buttons)
  # update_total_questions_quantity()
  

def on_answer_button_clicked(b):
    global finalSelectedBtn, list_of_stats
    # iterate over question widgets and get user answers
    answerId = randomQuestionAndAnswer['correctAnswer']
    if answerId:
      correctAns = answers[int(answerId)-1]

    if currentSelectedAns is NULL:
      print("Please select answer")
      return
    list_of_stats[len(questionsHistory)-1] += 1
    # check if user's answer is correct
    if currentSelectedAns.strip() == correctAns.strip():
        update_points(3)
        update_total_correct_questions_quantity()
        print("Correct!")
        finalSelectedBtn.style.button_color = 'lightgreen'
        # start timer and refresh
        time.sleep(1.5)
        generate_new_question()
    else:
        update_points(-1)
        finalSelectedBtn.style.button_color = 'red'
        display(IPython.display.HTML('<br/><h4>Incorrect answer<br/>Try Again! You can do it!<h4/><br/>'))

# Update points and display them
def update_points(new_points):
    global points
    points += new_points
    display(IPython.display.Markdown(f"**New Points:** {points}"))
    correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
    total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
    display(correct_label)
    display(total_label)

def update_total_questions_quantity():
    global quantity_of_questions
    quantity_of_questions+=1

def update_total_correct_questions_quantity():
    global quantity_of_correct_questions
    quantity_of_correct_questions+=1

def insert_new_question_to_db(b):
  global offset
  # if success - return true]
  
  # Connect to DB
  try:
    # TODO: check validation (not empty, no repeats)
    data_to_upload = {
      'question' : tb_question.value,
      'answers' : [None, tb_answer1.value, tb_answer2.value, tb_answer3.value, tb_answer4.value],
      'correctAnswer' : cb_correctAnswerId.value
    }
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    result = FBconn.put("/Questions", name=f"{len(questionsFromDB)+offset+1}", data=data_to_upload)
    offset += 1
    print("Success")
    return True
  except Exception as e:
    print(f"Can't insert into DB\n{e}")
    return False


def show_form():
    """
    Present a form for adding new questions into DB.
    """
    global cb_correctAnswerId, tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, offset
    output.clear()
    cb_correctAnswerId = widgets.Dropdown(options=['1','2','3','4'],  description='Correct answer Number:',
        disabled=False, style=style)

    tb_question = widgets.Text(description='Question:', style=style,)
    tb_answer1 = widgets.Text(description='Answer #1:', style=style,)
    tb_answer2 = widgets.Text(description='Answer #2:', style=style,)
    tb_answer3 = widgets.Text(description='Answer #3:', style=style,)
    tb_answer4 = widgets.Text(description='Answer #4:', style=style,)
    
    btn_insert = widgets.Button(description='Insert', disabled=False, 
                            button_style='success', 
                            layout=Layout(margin="20px 0px 0px 140px"))
    btn_insert.on_click(insert_new_question_to_db)
    display(VBox([tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, cb_correctAnswerId, btn_insert]))
    show_go_back(create_menu_buttons)

def show_go_back(back_event):
  if btn_go_back != NULL:
    display(btn_go_back)
  btn_go_back.on_click(back_event)

def on_admin_clicked(b):
    """
    Handle validation of admin
    """
    output.clear()
    tb_password = widgets.Password(description='Password:',  style=style, placeholder='Insert password')
    tb_password.value = ""
    btn_submit_password = widgets.Button(description="Login", layout=Layout(margin="0px 0px 0px 155px"))

    def is_admin_password_valid(b):
      if tb_password.value == "123123":
        show_form()
      else:
        print("Invalid password")

    btn_submit_password.on_click(is_admin_password_valid)
    display(VBox([tb_password, btn_submit_password]))
    display(IPython.display.HTML(f"<br/><br/>"))
    show_go_back(create_menu_buttons)

def create_menu_buttons(b=NULL):
    """
    Present the main menu
    """
    output.clear()
    global btn_go_back, list_of_stats
    btn_questions = widgets.Button(description="Questions")
    btn_classes = widgets.Button(description="Classes")
    btn_help = widgets.Button(description="Help")
    btn_admin = widgets.Button(description="Add Questions (Admin only)", 
                               layout=Layout(width="200px"))
    btn_go_back = widgets.Button(description="Back", button_style="primary",layout=Layout(width="80px"))
    def on_questions_clicked(b):
        global list_of_stats
        list_of_stats = [0]*len(questionsFromDB)
        generate_new_question()


    
    def on_classes_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Classes page</h2>"))
        display(IPython.display.HTML(f"<h4>To gain access to our exceptional"+
        " classes, kindly get in touch with our customer service team to arrange payment.</h4>"))
        show_go_back(create_menu_buttons)

    
    def on_help_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Help page</h2>"))
        display(IPython.display.HTML(f"<h4>Answer as many questions as possible "+ 
          f"with the fewest attempts.<br/>Each correct answer is worth 3 points, "+
          f"but each incorrect answer will cost you 1 point</h4>"))
        display(IPython.display.HTML(f"<br/><br/><h4>Have Fun!</h4>"))
        show_go_back(create_menu_buttons)


    # Assign callbacks to buttons
    btn_questions.on_click(on_questions_clicked)
    btn_classes.on_click(on_classes_clicked)
    btn_help.on_click(on_help_clicked)
    btn_admin.on_click(on_admin_clicked)
    
    # Display Menu buttons
    display(VBox([btn_questions, btn_classes, btn_help, btn_admin]))


# Define callbacks
def on_start_clicked(b):
    output.clear()
    create_menu_buttons()
    

def start_game():
    # Create buttons
    btn_start = widgets.Button(description="Start Now")
    display(btn_start)
    btn_start.on_click(on_start_clicked)


##### MAIN #####

get_all_database()
start_game()



[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Output()